In [231]:
import numpy as np
import pandas as pd
import jieba
import jieba.posseg as pseg
import re
import matplotlib.pyplot as plt
from pylab import rcParams
from gensim.models import word2vec

%matplotlib inline

In [232]:
# Customizing plots with style 
rcParams['figure.figsize'] = 10, 5
rcParams['lines.linewidth'] = 2
plt.style.use('ggplot')

# 給一個關鍵字 tag，並從中找到相像關鍵字

In [233]:
# 資料載入
user_list = ["電腦王阿達", "笑波子", "Fixiphone-阿倫的愛瘋修"]
doc_types = ["post", "comment"]

comment_file_path = "data/big_data/pos/" + user_list[2] + "_" + doc_types[1] + "_1" + ".csv"
post_file_path = "data/big_data/pos/" + user_list[2] + "_" + doc_types[0] + "_1" + ".csv"
# 回文
comment_data = pd.read_csv(comment_file_path, encoding="utf8")
# 主文
post_data = pd.read_csv(post_file_path, encoding="utf8")

In [261]:
pd.read_csv("data/big_data/3c_post_0.csv", usecols=["POST_CONTENT"]).values.tolist()

[['3C少女'],
 ['加大版3C配件-行動電源保護收納盒 加大版3C配件-行動電源保護收納盒'],
 ['哩程信用卡2018-pchome 3c折價券，花旗pchome 哩程信用卡2018-pchome 3c折價券，花旗pchome 哩程信用卡2018-pchome 3c折價券，花旗pchome'],
 ['本身從事3c網拍一年想拓展更多款式3c產品例如:有認證的行動電源.藍牙耳機.藍牙喇叭等等3c批發商如有批發商想合作，請站內信，謝謝'],
 ['哽塞咳嗽發紺'],
 ['Messenger訊息提醒音效突然沒有了有人跟我一樣嗎？手機'],
 ['3c少女ㄚ'],
 ['猜一個3C產品'],
 ['有夠好玩的3C產品'],
 ['廢文工作3C理財語言'],
 ['3C王欸豪猛噢'],
 ['3C電容式螢幕觸控保暖針織手套 五指皆可觸控 3C電容式螢幕觸控保暖針織手套 五指皆可觸控'],
 ['請問各位水水們家中有哪些3c產品?平常所知道的有手機電腦電視，家中還有哪些3c產品呢？'],
 ['韓版 旅行3C收納包-防震包-硬碟包 韓版 旅行3C收納包-防震包-硬碟包'],
 ['有3C商品ㄇ 0.0'],
 ['玩3c產品玩到去掛眼科'],
 ['3C破壞神捏!!'],
 ['無何讓小朋友戒掉手機3C'],
 ['我！喜歡3C賣場的味道不知從何形容起但這味道聞起來真的他媽的超爽而且別的地方都聞不到唯有在3C賣場才有它 便是3C賣場的精魂！所以3C賣場的味道有卦否-- 其實我不知道電腦到底怎麼發出那種味道的 PCB'],
 ['我覺得在3C出現後很多人就都被3C綁架了我老公甚至說如果不用3C就不知道要做什麼我自己是有很多其他事可以做不過也總是被3C吸住然後時間就不見了 Orz各位不用3C時都在做什麼呢或是沒有得用的時候--減肥好累歐..  _(:3」∠)_'],
 ['\\有8代魔人/'],
 ['\\有8代魔人/'],
 ['好多黑畫面呀看到就想笑(*ﾟ∀ﾟ)'],
 ['可悲父母用iPad帶小孩難怪智商那麼低整天抱著平板不放跑去當偶像'],
 ['我的快充線是怎麼了呢最近都是充電中而已 兩面都是這樣'],
 ['請問，哪邊有二手3c新莊收購的？'],
 ['請問下，哪邊有二手3c收購的？'],
 ['請問大家，哪邊有3c回收的？'],
 ['請問一下大家，哪邊

In [234]:
# 3c comment data
comment_data = pd.read_csv("data/big_data/3c_comment_0.csv", encoding="utf8")
# 3c post data
post_data = pd.read_csv("data/big_data/3c_post_0.csv", encoding="utf8")

In [235]:
print(comment_data.shape)
print(post_data.shape)

(500000, 11)
(500, 11)


In [236]:
# 找到回文特定的作者，文章 title
comment_data.loc[(comment_data["POST_AUTHOR"]=="Fixiphone-阿倫的愛瘋修") & (comment_data["POST_TITLE"].str.contains("iphone")),:]

,URL,POST_BOARD,POST_AUTHOR,POST_TITLE,PUSH_AUTHOR,PUSH_STATE,POST_CONTENT,POST_DATE,POST_TIME,POST_IP,PUSH_NO


In [237]:
# post 文章
post_data.loc[:,["POST_TITLE", "POST_CONTENT"]]

,POST_TITLE,POST_CONTENT
0,3c少女XD,3C少女
1,加大版3C配件-行動電源保護收納盒?,加大版3C配件-行動電源保護收納盒 加大版3C配件-行動電源保護收納盒
2,哩程信用卡2018-pchome 3c折價券，花旗pchome?,哩程信用卡2018-pchome 3c折價券，花旗pchome 哩程信用卡2018-pcho...
3,[問題] 尋找相關3c批發,本身從事3c網拍一年想拓展更多款式3c產品例如:有認證的行動電源.藍牙耳機.藍牙喇叭等等3c...
4,3C,哽塞咳嗽發紺
5,3C,Messenger訊息提醒音效突然沒有了有人跟我一樣嗎？手機
6,那個,3c少女ㄚ
7,美國妹子,猜一個3C產品
8,小蛙玩3C (1P),有夠好玩的3C產品
9,追蹤的板,廢文工作3C理財語言


In [248]:
# 主文長度排序
post_data["POST_CONTENT"].str.len().sort_values(ascending=False).head(5)

217    2106
188    2106
196    2106
201    1888
195    1844
Name: POST_CONTENT, dtype: int64

In [249]:
# 資料前處理
post_data = post_data.dropna()
comment_data = comment_data.dropna()
post_content_list = post_data["POST_CONTENT"].values.tolist()

In [240]:
post_content_list = comment_data["POST_CONTENT"].values.tolist()

In [266]:
post_content_list

['3C少女',
 '加大版3C配件-行動電源保護收納盒 加大版3C配件-行動電源保護收納盒',
 '哩程信用卡2018-pchome 3c折價券，花旗pchome 哩程信用卡2018-pchome 3c折價券，花旗pchome 哩程信用卡2018-pchome 3c折價券，花旗pchome',
 '本身從事3c網拍一年想拓展更多款式3c產品例如:有認證的行動電源.藍牙耳機.藍牙喇叭等等3c批發商如有批發商想合作，請站內信，謝謝',
 '哽塞咳嗽發紺',
 'Messenger訊息提醒音效突然沒有了有人跟我一樣嗎？手機',
 '3c少女ㄚ',
 '猜一個3C產品',
 '有夠好玩的3C產品',
 '廢文工作3C理財語言',
 '3C王欸豪猛噢',
 '3C電容式螢幕觸控保暖針織手套 五指皆可觸控 3C電容式螢幕觸控保暖針織手套 五指皆可觸控',
 '請問各位水水們家中有哪些3c產品?平常所知道的有手機電腦電視，家中還有哪些3c產品呢？',
 '韓版 旅行3C收納包-防震包-硬碟包 韓版 旅行3C收納包-防震包-硬碟包',
 '有3C商品ㄇ 0.0',
 '玩3c產品玩到去掛眼科',
 '3C破壞神捏!!',
 '無何讓小朋友戒掉手機3C',
 '我！喜歡3C賣場的味道不知從何形容起但這味道聞起來真的他媽的超爽而且別的地方都聞不到唯有在3C賣場才有它 便是3C賣場的精魂！所以3C賣場的味道有卦否-- 其實我不知道電腦到底怎麼發出那種味道的 PCB',
 '我覺得在3C出現後很多人就都被3C綁架了我老公甚至說如果不用3C就不知道要做什麼我自己是有很多其他事可以做不過也總是被3C吸住然後時間就不見了 Orz各位不用3C時都在做什麼呢或是沒有得用的時候--減肥好累歐..  _(:3」∠)_',
 '\\有8代魔人/',
 '\\有8代魔人/',
 '好多黑畫面呀看到就想笑(*ﾟ∀ﾟ)',
 '可悲父母用iPad帶小孩難怪智商那麼低整天抱著平板不放跑去當偶像',
 '我的快充線是怎麼了呢最近都是充電中而已 兩面都是這樣',
 '請問，哪邊有二手3c新莊收購的？',
 '請問下，哪邊有二手3c收購的？',
 '請問大家，哪邊有3c回收的？',
 '請問一下大家，哪邊有3c收購的？',
 '請問下，哪邊有回收3c的？',
 '請問一下大家，哪邊有收購3c的？',
 '請問下，哪邊

In [263]:
print(len(post_content_list))

500


In [186]:
# 使用其他文檔實驗
with open("data/ref_text_tw.txt", "r", encoding="utf-8") as content:
    post_content_list = [line.strip().replace(' ', '') for line in content]    

In [242]:
# 指定濾掉的詞性，並過濾掉其他詞
def filter_part_speech(pos_list, part_speech_list):
    return list(filter(lambda x: x.flag not in pos_list, part_speech_list))

In [264]:
# 用來存放分詞後的結果
preprocessed_documents = []
# 分詞與詞性對應
part_speech_mapping = {}
# stopword
with open("data/jieba_dict/stopwords.txt") as stop_words:
    stop_word_list = [stop_word.strip() for stop_word in stop_words]
# 支援繁體中文較好的詞庫
jieba.set_dictionary("data/jieba_dict/dict.txt.big")
for index, document in enumerate(post_content_list[:30000], 0):
    # 只取中文
    try:
        document = "".join(re.findall(r"[\u4e00-\u9fa5]+", document))
    except Exception as e:
        print("{}, index {}".format(str(e), index))
    
    part_speech_list = list(pseg.cut(document))
    # 去除保留字
    part_speech_list = list(filter(lambda x: x.word not in stop_word_list, part_speech_list))
    # 篩選字詞
    part_speech_list = filter_part_speech(['n', 'x', 'n', 'ng', 'nr', 'ns'], part_speech_list)
    preprocessed_document = []
    for part_speech in part_speech_list:
        preprocessed_document.append(part_speech.word)
        part_speech_mapping[part_speech.word] = part_speech.flag
    preprocessed_documents.append(preprocessed_document)

Building prefix dict from /Users/Mark1002/Desktop/project/python/nlp-experiment/data/jieba_dict/dict.txt.big ...
Loading model from cache /var/folders/dw/m2zgs87j3x19nl8mnfy3fs8c0000gn/T/jieba.ud2b054c4d13e51557150f7d36ba5f4d0.cache
Loading model cost 1.551 seconds.
Prefix dict has been built succesfully.


In [190]:
part_speech_mapping

{'一一': 'm',
 '一下': 'm',
 '一些': 'm',
 '一代': 'm',
 '一位': 'm',
 '一包': 'm',
 '一台': 'm',
 '一堆': 'm',
 '一天': 'm',
 '一定': 'd',
 '一年': 'm',
 '一手': 'm',
 '一月': 'm',
 '一桌': 'm',
 '一次': 'm',
 '一生': 'm',
 '一直': 'd',
 '一罐': 'm',
 '一起': 'm',
 '三只手': 'i',
 '三年': 'm',
 '三手': 'm',
 '三星': 'nz',
 '三段': 'm',
 '三碗': 'm',
 '三立': 'm',
 '三重': 'm',
 '上班': 'v',
 '上面': 'f',
 '下': 'f',
 '下午': 't',
 '下去': 't',
 '下手': 'v',
 '下玩下': 'v',
 '下班': 'v',
 '不乖': 'd',
 '不出': 'v',
 '不到': 'v',
 '不及': 'c',
 '不好': 'd',
 '不想': 'v',
 '不放': 'v',
 '不明': 'v',
 '不爽': 'a',
 '不用': 'v',
 '不知': 'v',
 '不高': 'v',
 '世代': 't',
 '中': 'f',
 '中信': 'nz',
 '中正': 'nz',
 '主': 'b',
 '主要': 'b',
 '之前': 'f',
 '之用': 'r',
 '乖': 'a',
 '乖乖': 'd',
 '亂': 'd',
 '了到': 'v',
 '了去': 'v',
 '了台': 'v',
 '了念': 'v',
 '了恩': 'v',
 '了拜': 'v',
 '了收': 'v',
 '予以': 'v',
 '二手': 'm',
 '二手交易': 'nz',
 '五六年': 'm',
 '五年': 't',
 '五指': 'j',
 '五次': 'm',
 '亦': 'd',
 '亦可': 'v',
 '享': 'v',
 '人教': 'nz',
 '今天': 't',
 '今年': 't',
 '今晚': 't',
 '付款': 'v',
 '代': 'q',
 '代壞': 'v',
 '以上': 'f',
 '

In [253]:
model = word2vec.Word2Vec(preprocessed_documents, min_count=3, window=20, sg=1)

In [254]:
model.wv.most_similar("超值", topn=50)

/Users/Mark1002/anaconda/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('免運', 0.9996582269668579),
 ('一次', 0.9995408058166504),
 ('項', 0.9995300769805908),
 ('亦可', 0.999374270439148),
 ('直接', 0.9988524913787842),
 ('碟', 0.9984917640686035),
 ('限定', 0.9984117150306702),
 ('大同', 0.9983258247375488),
 ('轉', 0.9975271224975586),
 ('億', 0.9965912103652954),
 ('請', 0.9957769513130188),
 ('背蓋', 0.9951934814453125),
 ('片', 0.9948363304138184),
 ('記', 0.9946683645248413),
 ('座', 0.994192361831665),
 ('卡元', 0.9941567182540894),
 ('天外', 0.9921872615814209),
 ('限量版', 0.9921747446060181),
 ('使用', 0.9911432266235352),
 ('用過', 0.9893280863761902),
 ('提供', 0.9878865480422974),
 ('買', 0.9878697395324707),
 ('以上', 0.9875352382659912),
 ('接收', 0.985211968421936),
 ('全', 0.9848045110702515),
 ('附', 0.9842004776000977),
 ('全新', 0.9826323390007019),
 ('加送', 0.9814814925193787),
 ('站內', 0.9814186692237854),
 ('除是', 0.976245641708374),
 ('米', 0.9745671153068542),
 ('開', 0.9734057784080505),
 ('未', 0.973349928855896),
 ('條', 0.9714839458465576),
 ('拆封', 0.9693443775177002),
 ('目

In [255]:
model.wv.vectors.shape

(174, 100)